## Download

In [3]:
import requests
import pathlib
import patoolib
import glob
import os

def download_file(url, dir, name):
    local_filename = dir / name
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename

In [4]:
from tqdm.notebook import tqdm

# 10 - 30
ids = ["4264835", "4264834", "4264838", "4264836", "4264839", "4266983", "4266984", "4267037", "4267050", "4266987", "4266973", "4267013", "4266962", "4267008", "4267023", "4267010", "4264284", "4266985", "4267040", "4267038", "4266966", "4266997", "4266963", "4266976", "4264837", "4264833", "4267026", "4267005", "4267006", "4266982", "4264297", "4266992", "4266972", "4266993", "4266970", "4267022", "4267033", "4267018", "4267000", "4267009", "4267012", "4267007", "4267047", "4266969", "4266967", "4267003", "4266995", "4267039", "4266975", "4267017", "4266965", "4266994", "4267019", "4266986", "4267043", "4267020", "4267001", "4267032", "4266960", "4266996", "4266978", "4267025", "4267004", "4266998", "4266990", "4267016", "4267041", "4267002", "4266961", "4266977", "4266980", "4266974", "4267028", "4266968", "4267021", "4267015", "4266964", "4267045", "4267049", "4266979", "4267035", "4267030", "4266988", "4267024", "4267011", "4267029", "4267036", "4267042", "4267014", "4267031", "4266971", "4267046", "4266991", "4267027", "4267048", "4267044", "4266999", "4267034", 
    "4411471", "4411469", "4411475", "4411476", "4411482", "4411558", "4411559", "4411557", "4411796", "4411560", "4411563", "4411562", "4411561", "4411564", "4411565", "4411566", "4411794", "4411797", "4411407", "4411407", "4411408", "4411406", "4411405", "4411417", "4411414", "4411415", "4411416", "4411420", "4411419", "4267523", "4410886", "4411396", "4411397", "4411398", "4411395", "4411402", "4411401", "4411403", "4411404", "4411426", "4411425", "4411424", "4411423", "4411428", "4411430", "4411427", "4411429", "4411433", "4411432", "4410826", "4410824", "4410825", "4410829", "4410828", "4410827", "4410832", "4410830", "4410833", "4410831", "4410836", "4410837", "4410835", "4410834", "4410838", "4410840", "4410859", "4410856", "4410857", "4410858", "4267535", "4409574", "4409575", "4409573", "4410087", "4410088", "4410086", "4410401", "4410402", "4410400", "4410839", "4404145", "4404449", "4404146", "4404150", "4404148", "4404151", "4404149", "4404366", "4404147", "4404367",

]

for id in tqdm(ids):
    url = f"https://dataverse.harvard.edu/api/access/datafile/{id}"
    name = url.split("/")[-1]
    path = "./data/" + name
    if os.path.exists(path):
        continue
    print(f"downloading {name}")
    download_file(url=url, dir=pathlib.Path("./downloaded"),
                  name=name + ".rar")
    print(f"extracting {name}")
    if not os.path.exists(path):
        os.mkdir(path)
    patoolib.extract_archive(pathlib.Path("./downloaded") 
                             / (name + ".rar"),
                             outdir="./data/" + name)
    os.rename(glob.glob("./data/" + name + "/*")[0], "./data/" + name + "/ct")
    os.remove(pathlib.Path("./downloaded") / (name + ".rar"))
%slack

  0%|          | 0/188 [00:00<?, ?it/s]

downloading 4264835
extracting 4264835
patool: Extracting downloaded/4264835.rar ...
patool: running /usr/bin/unrar x -- /home/mfujita/Documents/research/downloaded/4264835.rar
patool:     with cwd='./data/4264835'
patool: ... downloaded/4264835.rar extracted to `./data/4264835'.
downloading 4264834
extracting 4264834
patool: Extracting downloaded/4264834.rar ...
patool: running /usr/bin/unrar x -- /home/mfujita/Documents/research/downloaded/4264834.rar
patool:     with cwd='./data/4264834'
patool: ... downloaded/4264834.rar extracted to `./data/4264834'.
downloading 4264838
extracting 4264838
patool: Extracting downloaded/4264838.rar ...
patool: running /usr/bin/unrar x -- /home/mfujita/Documents/research/downloaded/4264838.rar
patool:     with cwd='./data/4264838'
patool: ... downloaded/4264838.rar extracted to `./data/4264838'.
downloading 4264836
extracting 4264836
patool: Extracting downloaded/4264836.rar ...
patool: running /usr/bin/unrar x -- /home/mfujita/Documents/research/dow

In [24]:
import glob

for idx, file in enumerate(glob.glob("./data/*")):
    if '__' in file:
        continue
    # os.remove(file + "/drr.pickle")
    # os.remove(file + "/fbp.pickle")
    # print(file)
    if len(glob.glob(file + "/ct/*")) == 1:
        for folder in glob.glob(file + "/ct/*"):
            print(folder)
            for img in glob.glob(folder + "/*.dcm"):
                print(img)
                imgname = img.split("/")[-1]
                print(file + "/ct/" + imgname)
                os.rename(img, file + "/ct/" + imgname)
            for img in glob.glob(folder + "/*"):
                if not ".dcm" in img:
                    # print(img)
                    imgname = img.split("/")[-1]
                    print(file + "/ct/" + imgname + ".dcm")
                    os.rename(img, file + "/ct/" + imgname + ".dcm")
    if len(glob.glob(file + "/ct")) == 0 or not os.path.isdir(file + "/ct"):
        os.remove(file + "/ct")
        os.mkdir(file + "/ct")
        for img in glob.glob(file + "/*.dcm"):
            imgname = img.split("/")[-1]
            print(img)
            print(file + "/ct/" + imgname)
            # os.rename(img, file + "/ct/" + imgname)
        # print(file)

## DRR, FBP, Resize

In [1]:
import importlib
import imageio.v2 as imageio
from patient import patient
import glob

# https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/6ACUZJ

from tqdm.notebook import tqdm
for vol_path in tqdm(glob.glob("./data/*")):
    if "__" in vol_path:
        continue
    vol = vol_path.split("/")[-1]
    # print(vol)
    p = patient.patient(name=vol)

# imageio.mimsave(f"pics/drr.gif", p.drr.img)
%slack

  0%|          | 0/188 [00:00<?, ?it/s]

DRR:   0%|          | 0/450 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.51 GiB (GPU 0; 11.76 GiB total capacity; 7.30 GiB already allocated; 1.50 GiB free; 8.19 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import imageio.v2 as imageio
import cupy as cp
from patient import patient

p = patient.patient(name="4410088")

# p.generate_drr(cont=False)

# p.generate_drr(cont=False)
# p.generate_posdrr(cont=False)

for idx, img in enumerate(p.drr.img):
    new_img = img
    new_img[new_img > 255] = 255
    p.drr.img[idx] = new_img.astype("uint8")
    # print(cp.max(cp.ravel(p.drr.img[idx])), cp.min(cp.ravel(p.drr.img[idx])))

p.generate_fbp(load_all=False)

imgs = [cp.asnumpy(img) for img in p.drr.img]
imageio.mimsave("drr.gif", imgs)

imgs = [cp.asnumpy(img) for img in p.posdrr.img]
imageio.mimsave("posdrr.gif", imgs)

FBP:   0%|          | 0/400 [00:00<?, ?it/s]

TypeError: patient.generate_fbp() got an unexpected keyword argument 'cont'

In [ ]:
a = 200
imageio.imsave("test3.png", cp.asnumpy(p.get_equiv_fbp(a)))
imageio.imsave("test4.png", cp.asnumpy(p.ct.img[a]))
%slack

In [ ]:
import glob, os, datetime

for vol in glob.glob("./data/*"):
    for file in glob.glob(vol + "/*"):
        if ".pickle" in file:
            os.remove(file)
            # print(file)

## Pix2Pix Prepare

In [6]:
from pix2pix.datasets import combine_ct_and_fbp
combine_ct_and_fbp.prep("./pix2pix/datasets/ctfbp")
%slack


  0%|          | 0/131 [00:00<?, ?it/s]

4410857
4266993
4267031
4410835
4267032
4411407
4410858
4410859
4411416
4410828
4410837
4267027


4267027:   0%|          | 0/90 [00:00<?, ?it/s]

4410838
4404366
4411557
4410402
4409575
4410827
4411429
4411469
4411414
4411471
4266990
4266973
4266978
4267000


4267000:   0%|          | 0/90 [00:00<?, ?it/s]

...

4267022:   0%|          | 0/90 [00:00<?, ?it/s]

4410886


4410886:   0%|          | 0/90 [00:00<?, ?it/s]

4411428


4411428:   0%|          | 0/90 [00:00<?, ?it/s]